## Modules and data

### Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import string
from nltk.corpus import stopwords
from sklearn.decomposition import NMF, TruncatedSVD, PCA
import matplotlib.pyplot as plt 
from sklearn import model_selection
from nltk.stem import PorterStemmer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model, svm, naive_bayes, neighbors, ensemble
from sklearn.model_selection import cross_val_score, cross_validate, KFold

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 1000)
%matplotlib inline

### Headlines data

In [2]:
headlines_df = pd.read_csv('../data/external/labelled_headlines_data.csv')

headlines_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who k

### Sentiment data

In [3]:
sentiment_df = pd.read_csv('../data/processed/sentiment_data.csv', index_col = 'Date').loc[:,'Subjectivity':'Negative']

sentiment_df.head()

,Subjectivity,Objectivity,Positive,Neutral,Negative
Date,,,,,
2008-08-08,75.000000,25.000000,18.750000,25.000000,56.250000
2008-08-11,83.333333,16.666667,41.666667,16.666667,41.666667
2008-08-12,56.250000,43.750000,18.750000,43.750000,37.500000
2008-08-13,38.461538,61.538462,15.384615,61.538462,23.076923
2008-08-14,45.454545,54.545455,36.363636,54.545455,9.090909


### Dow Jones Industrial Average (DJIA) data

In [4]:
djia_df = pd.read_csv('../data/external/dow_jones_data.csv', index_col = 'Date')

djia_df.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


# Preparation

## Shifting

In [5]:
# CHOSE SHIFT HERE
n_shift = 1

### Shift headlines data

In [6]:
shifted_headlines_df = headlines_df
shifted_headlines_df.Label = shifted_headlines_df.Label.shift(periods=-n_shift)

shifted_headlines_df.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1984,2016-06-27,1.0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,Weapons shipped into Jordan by the CIA and Sau...,Angela Merkel said the U.K. must file exit pap...,In a birth offering hope to a threatened speci...,Sky News Journalist Left Speechless As Leave M...,Giant panda in Macau gives birth to twins,Get out now: EU leader tells Britain it must i...,Sea turtle 'beaten and left for dead' on beach...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
1985,2016-06-28,1.0,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,UK's Osborne: 'Absolutely' going to have to cu...,'Do not let Scotland down now' : Scottish MEP ...,British pound could hit history-making dollar ...,"Merkel vows to strengthen EU, tells UK no 'che...","""Ryanair will not deploy new aircraft on route...","People, ever more greedy and stupid, destroy t...",Siemens freezes new UK wind power investment f...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
1986,2016-06-29,1.0,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,Iceland historian Johannesson wins presidentia...,99-Million-Yr-Old Bird Wings Found Encased in ...,A chatbot programmed by a British teenager has...,The Philippine president-elect said Monday he ...,Former Belgian Prime Minister ridicules Nigel ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Islamic State bombings in southern Yemen kill ...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
1987,2016-06-30,1.0,Jamaica proposes marijuana dispensers for tour...,Stephen Hawkin

## Cleaning

In [7]:
# CHOOSE CLEANING PARAMETERS HERE
n_headlines = 5
min_freq = 0.01
max_freq = 0.99 
ngram_tuple = (1,2)

### Cleaning function

In [8]:
def clean_news(in_df, num_stories=10, min_df=0.01, max_df=0.99, ngram_range=(1,1), vectorizer=CountVectorizer):

    # set columns
    columns = in_df.loc[:,'Top1':'Top25'].columns.tolist()

    # strip, remove punctuation, make lower case, and remove numbers
    for i in columns:
        in_df[i] = (in_df[i].apply(lambda x: (str(x)).strip())
                      .apply(lambda x: x.lstrip('b'))
                      .apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower()))
                      .apply(lambda x: re.sub('\w*\d\w*', '', x))
                )

    # concatenate the specified number of columns into one column
    temp_dict = {}
    for i in range(in_df.shape[0]):
        temp_string = ''
        for j in columns[0:num_stories]:
            temp_string = temp_string + ' ' + in_df.loc[i, j]

        temp_dict[in_df.loc[i, 'Date']] = [temp_string]

    text_df = pd.DataFrame(temp_dict).T.rename(columns={0: 'text'})

    # instantiate count CountVectorizer
    trans = vectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    X = trans.fit_transform(text_df.text)

    # create final document to feature matrix for modelling
    doc_feat_mat = pd.DataFrame(X.toarray(), columns=trans.get_feature_names(), index = text_df.index)
    doc_feat_mat.index.name = 'date'
    doc_feat_mat = doc_feat_mat.merge(in_df[['Date','Label']], left_on='date', right_on='Date')
    doc_feat_mat.set_index('Date', inplace=True)
    doc_feat_mat = doc_feat_mat[ ['Label'] + [ col for col in doc_feat_mat.columns if col != 'Label' ] ]
    
    return doc_feat_mat

### Clean shifted headlines data

In [9]:
clean_df = clean_news(shifted_headlines_df, num_stories=n_headlines, min_df=min_freq, max_df=max_freq, ngram_range=ngram_tuple, vectorizer=TfidfVectorizer)

clean_df.shape

(1989, 1161)

## Merging

### Merge shifted headlines data and shifted sentiment data

In [10]:
# # NOT INCLUDING DJIA
# merged_df = clean_df.merge(sentiment_df, left_on=clean_df.index, right_on=sentiment_df.index)
# merged_df.set_index('key_0', inplace=True)
# merged_df.index.name = 'Date'
# merged_df = merged_df[ ['Label'] + [ col for col in merged_df.columns if col != 'Label' ] ]

# merged_df.shape

In [11]:
# INCLUDING DJIA
merged_temp_df = clean_df.merge(sentiment_df, left_on=clean_df.index, right_on=sentiment_df.index)
merged_temp_df.set_index('key_0', inplace=True)

merged_df = merged_temp_df.merge(djia_df, left_on=merged_temp_df.index, right_on=djia_df.index)
merged_df.set_index('key_0', inplace=True)

merged_df.index.name = 'Date'
merged_df = merged_df[ ['Label'] + [ col for col in merged_df.columns if col != 'Label' ] ]

merged_df.head()

,Label,able,abortion,abuse,access,according,account,accounts,accused,accuses,act,acta,action,activist,activists,acts,actually,admits,afghan,afghanistan,africa,african,age,agency,agents,ago,agreement,ahead,aid,air,airport,al,al jazeera,alive,allegations,alleged,allegedly,allow,allowed,allowing,allows,amazon,ambassador,america,american,americans,amid,amnesty,amp,ancient,animals,announced,announces,anonymous,appeal,approved,approves,arab,arabia,area,armed,arms,army,arrest,arrested,arrests,article,ask,asked,asks,assange,assault,asylum,attack,attacked,attacks,australia,australian,australias,authorities,available,avoid,away,babies,baby,bad,ban,bank,bankers,banking,banks,banned,banning,bans,battle,bay,bbc,bbc news,beat,beaten,begin,beijing,believe,believed,berlin,best,better,bid,big,biggest,billion,billions,bin,birth,black,blair,block,blocked,blood,bodies,body,bomb,bombs,border,borders,born,boy,boys,bp,brazil,brazilian,break,breaking,breaks,bring,britain,britains,british,brother,build,building,built,burned,bush,business,buy,called,calling,calls,came,cameron,camp,campaign,canada,canadas,canadian,canadians,cancer,cannabis,capital,captured,car,care,carrying,cars,case,cases,cash,catholic,catholic church,caught,cause,caused,cell,censorship,cent,central,century,ceo,change,charge,charged,charges,chemical,chief,child,children,china,chinas,chinese,christian,church,cia,cities,citizens,city,civil,civilians,claim,claims,clean,clear,climate,climate change,close,cnn,coast,collapse,combat,come,comes,coming,commercial,community,companies,company,computer,conditions,conference,confirmed,confirms,conflict,congo,considered,content,continue,control,controversial,convicted,cops,copyright,corruption,cost,council,countries,country,countrys,couple,court,court rules,crackdown,crash,crime,crimes,criminal,crisis,cuba,cup,current,cut,...,services,set,settlement,settlements,settlers,seven,sex,sexual,ship,shoot,shooting,shot,shot dead,showing,shows,shut,sign,single,site,sites,small,snowden,social,society,solar,sold,soldier,soldiers,son,soon,source,south,south african,south korea,south korean,southern,soviet,space,spain,spanish,special,speech,spent,spill,spread,spy,spying,square,staff,stand,start,started,state,states,stay,step,stephen,stolen,stop,storm,story,street,streets,strike,strikes,strip,student,students,study,sue,suffering,suggests,suicide,sunday,supply,support,supreme,supreme court,surveillance,suspect,suspected,sweden,swedish,swiss,syria,syrian,taken,takes,taking,taliban,talks,tax,taxes,team,tells,territory,terror,terrorism,terrorist,terrorists,test,text,thats,theres,theyre,theyve,think,thought,thousands,threat,threaten,threatens,thursday,time,times,today,told,tony,took,toronto,torture,tortured,total,tourists,town,tpp,trade,traffic,treated,treatment,trial,tried,tries,troops,try,trying,tsunami,tuesday,turkey,turkish,turn,turned,turns,tv,twitter,uganda,uk,uk government,ukraine,ukrainian,union,united,united nations,united states,university,urges,usa,use,used,users,uses,using,vatican,venezuela,victim,victims,video,view,village,violence,violent,virus,visit,vladimir,vladimir putin,vote,voted,votes,wall,want,wanted,wants,war,war crimes,war drugs,warned,warning,warns,washington,watch,water,waters,way,weapons,wearing,web,website,websites,wednesday,week,weeks,west,west bank,western,whistleblower,white,wife,wikileaks,wikileaks founder,win,wind,wins,woman,women,wont,work,workers,working,world,world cup,world war,worlds,worlds largest,worldwide,worst,worth,wounded,wrong,year,year old,years,years ago,years prison,yemen,young,youre,youtube,zealand,zimbabwe,Subjectivity,Objectivity,Positive,Neutral,Negative,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

### Drop null columnes

In [12]:
merged_df.dropna(inplace=True)

merged_df.shape

(1985, 1172)

## Dimensionality reduction

In [13]:
# CHOOSE DIMENSIONALITY REDUCTION PARAMETERS HERE
n_dimension = 5
reduction_model = NMF   # NMF, PCA, TruncatedSVD

### Dimensionality reduction function

In [14]:
# function to give document to topics relationships
def dimension_reduce(in_df, topic_names, n_components = 5, model=NMF):

    feats_df = in_df.drop(columns=['Label'])

    # instantiate model
    the_model = model(n_components = n_components, random_state=10)
    if model == PCA:
        values = the_model.fit_transform(np.array(feats_df)-np.array(feats_df).mean())
    else:
        values = the_model.fit_transform(feats_df)

    top_df = pd.DataFrame(values, index = feats_df.index, columns = topic_names)
    top_df = top_df.merge(clean_df.reset_index()[['Date','Label']], left_on='Date', right_on='Date')
    top_df.set_index('Date', inplace=True)
    top_df = top_df[ ['Label'] + [ col for col in top_df.columns if col != 'Label' ] ]
    
    # make document to topics dataframe
    return top_df

### Reduce data

In [15]:
topic_names = [f'dim_{i}' for i in range(1, n_dimension+1)]

In [16]:
reduced_df = dimension_reduce(merged_df, topic_names, n_components = n_dimension, model=reduction_model)

reduced_df.head()

,Label,dim_1,dim_2,dim_3,dim_4,dim_5
Date,,,,,,
2008-08-08,1.0,2325.359696,13.847980,0.300511,1.559890,0.000000
2008-08-11,0.0,2001.515965,15.433444,0.165923,0.360307,0.000000
2008-08-12,0.0,1896.627307,13.651824,1.791386,0.805135,0.139378
2008-08-13,1.0,1994.523387,12.174138,1.390599,0.656323,0.851106
2008-08-14,1.0,1745.849861,13.451105,0.567002,0.775542,0.722763


# Modelling

In [17]:
accuracies = []

### Split

In [18]:
X = np.array(reduced_df.iloc[:, 1:])
y = np.array(reduced_df.loc[:, 'Label']).astype(float)
X, X_test, y, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=11)

### Scale

In [19]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

## Linear discriminant analysis (LDA)

### Working LDA

In [20]:
lda_model = lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X,y)
y_pred = lda_model.predict(X_test)

In [21]:
print("Precision = {}".format(precision_score(y_test, y_pred)))
print("Recall = {}".format(recall_score(y_test, y_pred)))
print("Accuracy = {}".format(accuracy_score(y_test, y_pred)))
print("ROC AUC = {}".format(roc_auc_score(y_test, y_pred)))
print("F1 = {}".format(f1_score(y_test, y_pred)))

Precision = 0.5923566878980892
Recall = 0.808695652173913
Accuracy = 0.5667506297229219
ROC AUC = 0.5211142931528248
F1 = 0.6838235294117647


In [22]:
accuracies.append(accuracy_score(y_test, y_pred).round(3))

### Solver tuning

### Shrinkage tuning

## Xgboost

### Working xgb

In [23]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X,y)
y_pred = (xgb_model.predict(X_test) > 0.5).astype(int)

[14:22:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
print("Precision = {}".format(precision_score(y_test, y_pred)))
print("Recall = {}".format(recall_score(y_test, y_pred)))
print("Accuracy = {}".format(accuracy_score(y_test, y_pred)))
print("ROC AUC = {}".format(roc_auc_score(y_test, y_pred)))
print("F1 = {}".format(f1_score(y_test, y_pred)))

Precision = 0.5411255411255411
Recall = 0.5434782608695652
Accuracy = 0.46851385390428213
ROC AUC = 0.45437386097370475
F1 = 0.5422993492407809


In [25]:
accuracies.append(accuracy_score(y_test, y_pred).round(3))

## Naive Bayes

### Gaussian naive Bayes

In [26]:
nb_model = naive_bayes.GaussianNB()
nb_model.fit(X, y)
y_pred = (nb_model.predict_proba(X_test)[:, 1] >= 0.39)

In [27]:
print("Precision = {}".format(precision_score(y_test, y_pred)))
print("Recall = {}".format(recall_score(y_test, y_pred)))
print("Accuracy = {}".format(accuracy_score(y_test, y_pred)))
print("ROC AUC = {}".format(roc_auc_score(y_test, y_pred)))
print("F1 = {}".format(f1_score(y_test, y_pred)))

Precision = 0.5947368421052631
Recall = 0.9826086956521739
Accuracy = 0.6020151133501259
ROC AUC = 0.5302265035147097
F1 = 0.7409836065573769


In [28]:
accuracies.append(accuracy_score(y_test, y_pred).round(3))

### Bernoulli naive Bayes

In [29]:
nb_model = naive_bayes.BernoulliNB()
nb_model.fit(X, y)
y_pred = (nb_model.predict_proba(X_test)[:, 1] >= 0.5)

In [30]:
print("Precision = {}".format(precision_score(y_test, y_pred)))
print("Recall = {}".format(recall_score(y_test, y_pred)))
print("Accuracy = {}".format(accuracy_score(y_test, y_pred)))
print("ROC AUC = {}".format(roc_auc_score(y_test, y_pred)))
print("F1 = {}".format(f1_score(y_test, y_pred)))

Precision = 0.5745454545454546
Recall = 0.6869565217391305
Accuracy = 0.5239294710327456
ROC AUC = 0.49317885967196046
F1 = 0.6257425742574257


In [31]:
accuracies.append(accuracy_score(y_test, y_pred).round(3))

### Threshold tuning

In [32]:
scores = {}

for i in range(0,100):   
    threshold = np.linspace(0,1,100)[i]
    
    nb_model = naive_bayes.GaussianNB()
    nb_model.fit(X, y)
    y_pred = (nb_model.predict_proba(X_test)[:, 1] >= threshold)     

    scores[threshold] = accuracy_score(y_test, y_pred)
    
scores

{0.0: 0.5793450881612091,
 0.010101010101010102: 0.5793450881612091,
 0.020202020202020204: 0.5793450881612091,
 0.030303030303030304: 0.5818639798488665,
 0.04040404040404041: 0.5818639798488665,
 0.05050505050505051: 0.5818639798488665,
 0.06060606060606061: 0.5818639798488665,
 0.07070707070707072: 0.5818639798488665,
 0.08080808080808081: 0.5818639798488665,
 0.09090909090909091: 0.5818639798488665,
 0.10101010101010102: 0.5818639798488665,
 0.11111111111111112: 0.5793450881612091,
 0.12121212121212122: 0.5793450881612091,
 0.13131313131313133: 0.5793450881612091,
 0.14141414141414144: 0.5793450881612091,
 0.15151515151515152: 0.5793450881612091,
 0.16161616161616163: 0.5793450881612091,
 0.17171717171717174: 0.5793450881612091,
 0.18181818181818182: 0.5793450881612091,
 0.19191919191919193: 0.5793450881612091,
 0.20202020202020204: 0.5793450881612091,
 0.21212121212121213: 0.5793450881612091,
 0.22222222222222224: 0.5818639798488665,
 0.23232323232323235: 0.5818639798488665,
 0.24

# Best result

In [33]:
accuracies

[0.567, 0.469, 0.602, 0.524]

Best is Guassian Naiive Bayes with 60.2